In [1]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient
import pandas as pd

In [6]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '429'
start = '2021-06-11T00:00:00Z'
end = '2021-06-25T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [7]:
def get_dataset():

    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', "api"]},
        'original_item.created_at': {'$gte': start, '$lte': end},
        'original_item.custom_fields': {"$ne":
            {
                "id" : 360040654351,
                "value" : None
            }
        }
    })
    
    file_path = os.path.expanduser('~/Downloads/sl_self_service_preds.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket["original_item"]
            j = {
                "ticket_id": i['id'], 
                "subject": i["subject"],
                "description": i["description"],
                "created_at": i["created_at"],
                "label": [x['value'] for x in i['custom_fields'] if x['id'] == 360040654351]
                }
                
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)

    print("total: ", count)


get_dataset()

100
200
300
400
500
600
700
total:  727


In [2]:
ticketlist = []
predictionlist = []
output = []

with open('sl_self_service_preds.json') as file:
	for obj in file:
		ticket = json.loads(obj)
		predictionlist.append(ticket)

with open('sl_self_service_cor.json') as file:
	for obj in file:
		ticket = json.loads(obj)
		ticketlist.append(ticket)

In [1]:
correct = []

with open('sl_self_service_preds.json') as file:
    for prediction in predictionlist:
        for ticket in ticketlist:
            if prediction['ticket_id'] == ticket['ticket_id'] and prediction['label'] == ticket['label']:
                tick = {
                    "id": prediction['ticket_id'],
                    'description': prediction['description'],
                    'label': prediction['label']
                }
                correct.append(tick)
                for item in correct:
                    print(item)
                    print("")

print(len(correct))
            

NameError: name 'predictionlist' is not defined

In [ ]:
for item in correct:
    print(item)
    print("")

print(len(correct))
            